In [1]:
%load_ext line_profiler
from line_profiler import profile

In [2]:
%load_ext line_profiler
import cv2
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import numpy
import math

# Globale Variablen und Funktionen außerhalb der Hauptfunktion
patch_size = 7
patch_half = int(patch_size/2)
defined = 0
source_flag = 128
undefined = 255

def sparse_l2_cost(source_patch, target_patch, image, mask):
    sx,sy = source_patch
    tx,ty = target_patch
    sum = 0.0
    for dy in range(-patch_half, patch_half+1):
        for dx in range(-patch_half, patch_half+1):
            if mask[sy+dy,sx+dx] == undefined:
                return None    
            if mask[ty+dy,tx+dx] == undefined:
                continue
            for channel in range(3):
                difference = image[sy+dy,sx+dx,channel] - image[ty+dy,tx+dx,channel]
                difference = difference*difference
                sum += difference
    return math.sqrt(sum)

@profile
def find_best_patch(image, source_region, target, mask): 
    target_x,target_y = target
    all_cost = []
    for source_x, source_y in source_region:
        cost = sparse_l2_cost((source_x,source_y), (target_x,target_y), image, mask)
        if not cost is None:
            all_cost.append((source_x,source_y, cost))
    all_cost = sorted(all_cost, key=lambda x: x[2], reverse=False)
    selected_x,selected_y,_ = all_cost[0]
    return selected_x,selected_y

def init_confidence(mask):
    confidence = numpy.zeros(mask.shape, dtype=numpy.float32)
    for y in range(mask.shape[0]):
        for x in range(mask.shape[1]):
            if mask[y,x] == undefined:
                confidence[y,x] = 0.0
            else:
                confidence[y,x] = 1.0           
    return confidence

def compute_confidence(mask, confidence, x, y):
    sum = 0.0
    for dy in range(-patch_half, patch_half+1):
        for dx in range(-patch_half, patch_half+1):
            if mask[y+dy,x+dx] == undefined:
                continue
            sum += confidence[y+dy,x+dx]
    return sum / (patch_size*patch_size)

def extract_fillfront(mask, confidence):
    fillfront = []
    for y in range(1, mask.shape[0]-1):
        for x in range(1, mask.shape[1]-1):
            if mask[y,x] != undefined:
                continue
            if (mask[y,x-1] != undefined) or (mask[y,x+1] != undefined) or (mask[y-1,x] != undefined) or (mask[y+1,x] != undefined):
                c = compute_confidence(mask, confidence, x, y)
                fillfront.append((x,y,c))
    return fillfront

def extract_source_region(mask):
    print("extracting source region")
    source_region = []
    for source_y in range(patch_half, mask.shape[0]-patch_half-1):
        for source_x in range(patch_half, mask.shape[1]-patch_half-1):
            patch_valid = True
            for dy in range(-patch_half, patch_half+1):
                for dx in range(-patch_half, patch_half+1):    
                    if mask[source_y+dy,source_x+dx] != source_flag:
                        patch_valid=False
            if patch_valid:
                source_region.append((source_x,source_y))
    return source_region

def display(axis, image, mask):
    display = image[:, :, ::-1]
    axis.imshow(display.astype(numpy.uint8))
    axis.set_xticks([])
    axis.set_yticks([])

def show_fillfront(fillfront, axis):
    x = [x for x,_,_ in fillfront]
    y = [y for _,y,_ in fillfront]
    c = [c for _,_,c in fillfront]
    axis.scatter(x, y, c=c, s=1, cmap='seismic')

def highlight_patch(axis, x, y):
    ox = x-patch_half
    oy = y-patch_half
    rectangle = Rectangle((ox,oy), patch_size, patch_size, edgecolor="red", fill=None)
    axis.add_patch(rectangle)

def update_image(image, source, target):
    source_x,source_y = source
    target_x,target_y = target
    for dy in range(-patch_half, patch_half+1):
        for dx in range(-patch_half, patch_half+1):    
            image[target_y+dy,target_x+dx] = image[source_y+dy,source_x+dx]

def update_mask(mask, source, target):
    source_x,source_y = source
    target_x,target_y = target
    for dy in range(-patch_half, patch_half+1):
        for dx in range(-patch_half, patch_half+1):    
            mask[target_y+dy,target_x+dx] = defined

def update_confidence(mask, confidence, source, target):
    source_x,source_y = source
    target_x,target_y = target
    new_confidence = numpy.zeros((patch_size, patch_size), dtype=numpy.float32)
    for dy in range(0, patch_size):
        for dx in range(0, patch_size):
            new_confidence[dy, dx] = compute_confidence(mask, confidence, target_x+dx-patch_half, target_y+dy-patch_half)
    confidence[target_y-patch_half:target_y+patch_half+1,target_x-patch_half:target_x+patch_half+1] = new_confidence

def perform_inpainting(image, mask, confidence, source, target):
    update_image(image, source, target)
    update_mask(mask, source, target)
    update_confidence(mask, confidence, source, target)

def main():
    image = cv2.imread("image_02.jpg").astype(numpy.float32)
    mask = cv2.imread("mask_02.png", cv2.IMREAD_GRAYSCALE)
    image[mask==255,:] = 0.0
    
    confidence = init_confidence(mask)
    source_region = extract_source_region(mask)
    fillfront = extract_fillfront(mask, confidence)
    
    if len(fillfront) > 0:
        fillfront = sorted(fillfront, key=lambda x: x[2], reverse=True)
        target_x,target_y,_ = fillfront[0]
        result = find_best_patch(image, source_region, (target_x,target_y), mask)
        return result



The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
%lprun -f find_best_patch main()

extracting source region


Timer unit: 1e-09 s

Total time: 4.6271 s
File: /var/folders/1n/s0sv22yn5mxbn5kqpvb2hyk40000gn/T/ipykernel_10340/3055113519.py
Function: find_best_patch at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
    31                                           @profile
    32                                           def find_best_patch(image, source_region, target, mask): 
    33         1       1000.0   1000.0      0.0      target_x,target_y = target
    34         1       1000.0   1000.0      0.0      all_cost = []
    35     81457   10916000.0    134.0      0.2      for source_x, source_y in source_region:
    36     81456 4581550000.0  56245.7     99.0          cost = sparse_l2_cost((source_x,source_y), (target_x,target_y), image, mask)
    37     81456    6113000.0     75.0      0.1          if not cost is None:
    38     81456    8953000.0    109.9      0.2              all_cost.append((source_x,source_y, cost))
    39         1   19564000.0    2e+07      0.4   